# Data cleaning

In [1]:
import pandas as pd
import numpy as np
import random
import json
from tensorflow.keras.models import Model, load_model
# import pycaret

# Replace double quotes with single quotes for moods JSON arrays
f = open('song_data.csv','r+')
text = f.read()
text = text.replace('"["', '"[?').replace('", "', '?,?').replace('"]"', '?]"') # Changed to "[$Happy$,$Sad$]" for easy replacing later
f.seek(0)
f.write(text)
f.close()

# Importing data
df = pd.read_csv('song_data.csv', index_col=0)
df.drop('uuid', axis=1, inplace=True)
df.dropna(inplace=True) # drop rows with nan values
for col in df.columns:
    if col not in ['id','isSkipped']:
        if col == 'moods':
            df[col] = df[col].apply(lambda x:x.replace('?','"')) # Replaces mood values back to the form ["Happy","Sad"] so it can be loaded by json
        df[col] = df[col].apply(json.loads)
df['activity'] = df.apply(lambda _: '', axis=1) # empty activity column
print('Number of samples: ', df.shape[0])
df.head()

Number of samples:  495


,gyroX,gyroY,gyroZ,accelX,accelY,accelZ,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,,,,,,,
1,"[499.30572509765625, 499.53460693359375, 0.198...","[1.861572265625, 2.49481201171875, 1.022338867...","[1.24359130859375, 1.2359619140625, 1.06048583...","[1.1943359375, 1.201171875, 1.1845703125, 1.18...","[1.1455078125, 1.1591796875, 1.1630859375, 1.1...","[3.68359375, 3.654296875, 3.6748046875, 3.6650...","[139.64, 138.36, 139.64, 140.28]","[30.50567626953125, 30.50567626953125, 30.5056...","[71.3134765625, 71.3134765625, 71.3134765625]","[Depressive, Atmospheric]",0,
2,"[498.1765747070313, 0.98419189453125, 1.579284...","[32.27996826171875, 14.7247314453125, 9.864807...","[497.9248046875, 496.368408203125, 494.9645996...","[2.9326171875, 2.9345703125, 2.728515625, 2.64...","[0.8466796875, 0.74609375, 0.865234375, 15.524...","[2.7548828125, 2.8037109375, 2.806640625, 3.34...","[123.24, 123.24, 139.32, 228.64]","[31.69403076171875, 31.69403076171875, 31.6940...","[67.05322265625, 67.05322265625, 67.0532226562...",[Depressive],0,
3,"[15.76995849609375, 10.65826416015625, 6.87408...","[488.36517333984375, 486.5798950195313, 496.92...","[3.86810302734375, 5.0811767578125, 498.947143...","[14.0107421875, 14.3212890625, 14.232421875, 1...","[14.96484375, 15.212890625, 15.275390625, 15.1...","[3.2021484375, 3.3291015625, 3.375, 3.35644531...","[256.08, 307.84000000000003, 315.2, 301.36, 30...","[32.21771240234375, 32.21771240234375, 32.2177...","[65.850830078125, 65.850830078125, 65.85083007...","[Passionate, Depressive]",0,
4,"[499.93896484375, 499.45068359375, 499.7482299...","[1.82342529296875, 2.74658203125, 1.8844604492...","[1.57928466796875, 1.434326171875, 1.365661621...","[1.7353515625, 1.708984375, 1.7333984375, 1.71...","[13.7841796875, 13.80078125, 13.7744140625, 13...","[2.8232421875, 2.8369140625, 2.8154296875, 2.8...","[127.08, 126.76, 125.48, 124.52]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...",[Elegant],1,
5,"[499.9465942382813, 0.03814697265625, 499.7482...","[2.01416015625, 1.77001953125, 1.7852783203125...","[1.1444091796875, 1.2359619140625, 1.129150390...","[0.4580078125, 0.4609375, 0.453125, 0.43847656...","[13.181640625, 13.1689453125, 13.1787109375, 1...","[2.6806640625, 2.693359375, 2.6875, 2.68652343...","[145.76, 144.48, 146.4, 144.8]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...","[Passionate, Depressive]",1,


In [2]:
data = df.iloc[0,:]
display(data)
# print(data['temp'])

gyroX        [499.30572509765625, 499.53460693359375, 0.198...
gyroY        [1.861572265625, 2.49481201171875, 1.022338867...
gyroZ        [1.24359130859375, 1.2359619140625, 1.06048583...
accelX       [1.1943359375, 1.201171875, 1.1845703125, 1.18...
accelY       [1.1455078125, 1.1591796875, 1.1630859375, 1.1...
accelZ       [3.68359375, 3.654296875, 3.6748046875, 3.6650...
optical                       [139.64, 138.36, 139.64, 140.28]
temp         [30.50567626953125, 30.50567626953125, 30.5056...
humidity         [71.3134765625, 71.3134765625, 71.3134765625]
moods                                [Depressive, Atmospheric]
isSkipped                                                    0
activity                                                      
Name: 1, dtype: object

In [3]:
# Filtering defective data

defective_ids = []
for idx,row in df.iterrows():
    # defective if temp array only has -40 values
    # defective if any humidity values are above 99.99
    if len([k for k in row['temp'] if k==-40]) == len(row['temp']) or \
    len([k for k in row['humidity'] if k>99.99]) == len(row['humidity']):
        defective_ids.append(idx)
    
    # if only some values are defective, keep the row, but remove defective values
    # remove -40 temp values and >99.99 humidity values
    elif (-40 in row['temp']) or len([k for k in row['humidity'] if k>99.99]) or (0 in row['optical']):
        df.at[idx,'temp'] = [k for k in row['temp'] if k!=-40]
        df.at[idx,'humidity'] = [k for k in row['humidity'] if k<99.99]
        
    # some gyro/accel data have 40 samples
    # take the last 30 samples for these rows
    for col in df.columns[:6]:
        if len(row[col]) > 30:
            df.at[idx,col] = row[col][-30:]

filtered_df = df[~df.index.isin(defective_ids)].copy() # .copy() to avoid warning
print('%d defective rows: ' % len(defective_ids),defective_ids)

23 defective rows:  [21, 22, 33, 214, 236, 238, 245, 246, 247, 248, 249, 250, 251, 252, 386, 387, 388, 389, 390, 391, 392, 393, 394]


In [4]:
motion_model_path = 'savedModel_stackedLSTM'
model = load_model(motion_model_path)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 128)           69120     
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 64)            49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               3300      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 134,547
Trainable params: 134,547
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Add activity from motion recognition model
x = [list(k) for k in filtered_df.iloc[:,:6].values]
x = np.array(x) # (num_samples, 6, 30)
x = np.array([k.T for k in x]) # reshape as (num_samples, 30, 6)

pred = model.predict(x)

activity_cats = np.array(['Running', 'Walking', 'Working']) # hardcoded categories from 'Physical Activity Classification.ipynb'
# filtered_df['activity'] = encoder_categories[np.argmax(pred, axis=1)]
filtered_df['activity'] = np.argmax(pred, axis=1)
filtered_df = filtered_df.iloc[:,6:].copy() # drop gyro and accel columns

filtered_df.sample(5)

,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,
6,"[208.16, 208.16, 205.92, 206.24, 206.56]","[32.49969482421875, 32.49969482421875, 32.4996...","[63.531494140625, 63.531494140625, 63.53149414...",[Atmospheric],1,2
176,"[1147.52, 497.28, 656, 1234.88]","[33.38592529296875, 33.38592529296875, 33.3859...","[74.310302734375, 74.310302734375, 74.31030273...",[Atmospheric],1,0
139,"[15.62, 16.92, 20.39, 43.02]","[32.0062255859375, 32.0062255859375, 32.006225...","[64.5751953125, 64.5751953125, 64.5751953125, ...","[Celebratory, Passionate]",1,1
553,"[262.96, 263.28000000000003, 263.6, 262.96, 26...","[33.355712890625, 33.355712890625, 33.35571289...","[60.97412109375, 60.97412109375, 60.9741210937...",[Depressive],1,2
423,"[39.47, 39.31, 38.91, 35.76]","[32.41912841796875, 32.41912841796875, 32.4191...","[71.7529296875, 71.7529296875, 71.7529296875]","[Aggressive, Passionate, Depressive]",0,2


In [6]:
# Obtain mean optical, temp and humidity values

for col in filtered_df.columns:
    if col not in ['moods','isSkipped','activity']:
        filtered_df[col] = filtered_df[col].apply(np.mean)
filtered_df.head()

,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,
1,139.480,30.505676,71.313477,"[Depressive, Atmospheric]",0,2
2,153.610,31.694031,67.053223,[Depressive],0,2
3,297.792,32.217712,65.850830,"[Passionate, Depressive]",0,2
4,125.960,32.429199,64.672852,[Elegant],1,2
5,145.360,32.429199,64.672852,"[Passionate, Depressive]",1,2


In [7]:
# One-hot encoding for moods

moods = []
for k in filtered_df['moods'].values:
    moods += list(k)
moods = np.unique(np.array(moods))
for mood in moods:
    mood_values = filtered_df['moods'].astype(str).str.contains(mood)
    filtered_df[mood] = mood_values
filtered_df.drop('moods', axis=1, inplace=True)
print('Added one-hot encoded columns for moods:')
filtered_df.head()

Added one-hot encoded columns for moods:


,optical,temp,humidity,isSkipped,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,,
1,139.480,30.505676,71.313477,0,2,False,False,True,False,True,False,False,False
2,153.610,31.694031,67.053223,0,2,False,False,False,False,True,False,False,False
3,297.792,32.217712,65.850830,0,2,False,False,False,False,True,False,True,False
4,125.960,32.429199,64.672852,1,2,False,False,False,False,False,True,False,False
5,145.360,32.429199,64.672852,1,2,False,False,False,False,True,False,True,False


In [8]:
# Invert mood boolean values based on "isSkipped"

for mood in moods:
    filtered_df[mood] = np.abs(filtered_df[mood] - filtered_df['isSkipped'])
filtered_df.drop('isSkipped', axis=1, inplace=True)
print('Invert mood values based on "isSkipped" boolean:')
filtered_df.head()

Invert mood values based on "isSkipped" boolean:


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
1,139.480,30.505676,71.313477,2,0,0,1,0,1,0,0,0
2,153.610,31.694031,67.053223,2,0,0,0,0,1,0,0,0
3,297.792,32.217712,65.850830,2,0,0,0,0,1,0,1,0
4,125.960,32.429199,64.672852,2,1,1,1,1,1,0,1,1
5,145.360,32.429199,64.672852,2,1,1,1,1,0,1,0,1


# Split into train/test datasets

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# label encoding for activity
# le = LabelEncoder()
# filtered_df['activity'] = le.fit_transform(filtered_df['activity'].values)
# display(filtered_df.head())

# split into training & testing
x = filtered_df.iloc[:,:4]
y = filtered_df.iloc[:,4:]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

print('Training (x, y): ', x_train.shape, y_train.shape)
print('Testing (x, y): ', x_test.shape, y_test.shape)

# create dfs for training and test data for easy prediction later
train_df = x_train.join(y_train)
test_df = x_test.join(y_test)

print('\nTrain:')
display(train_df.head())
print('Test:')
display(test_df.head())

Training (x, y):  (377, 4) (377, 8)
Testing (x, y):  (95, 4) (95, 8)

Train:


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
130,162.120,34.161377,56.451416,2,0,0,1,0,0,1,0,0
461,24.980,33.174438,59.014893,2,0,0,0,0,0,1,0,0
70,89.510,34.040527,74.023438,2,0,1,1,1,1,1,1,1
428,79.045,32.419128,71.752930,2,0,1,0,0,0,0,0,0
162,252.320,34.080811,59.002686,2,1,1,1,1,1,0,1,0


Test:


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
119,234.80,33.818970,58.093262,2,0,0,0,0,0,1,0,1
91,92.69,34.100952,75.988770,2,0,0,1,0,0,0,0,1
24,60.68,32.630615,61.993408,2,1,1,1,0,1,1,1,1
376,29688.32,38.139343,55.944824,1,0,0,0,0,1,0,0,0
207,5607.68,34.674988,70.166016,1,1,1,1,1,1,1,1,0


# SVM

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import svm

# model training
# since predict_proba only works with two classes at a time,
# eight svm models are trained for each mood and stored in the dictionary "svm_moods"
svms_std = {} # key:mood, value:svm model trained on that mood
params = {'C':[10e-3, 10e-2, 0.1, 1,10], 'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'decision_function_shape':('ovr', 'ovo')}
for mood in y_train.columns:
    svm_pipe_std = Pipeline([('scaler', StandardScaler()), 
                       ('svm', GridSearchCV(svm.SVC(max_iter=100000, probability=True), params)), ])
    svm_pipe_std.fit(x_train, y_train.loc[:,mood].values)
    svms_std[mood] = svm_pipe_std

C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre

In [11]:
svms_mm = {} # key:mood, value:svm model trained on that mood
params = {'C':[10e-3, 10e-2, 0.1, 1,10], 'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'decision_function_shape':('ovr', 'ovo')}
for mood in y_train.columns:
    svm_pipe_mm = Pipeline([('scaler', MinMaxScaler()), 
                       ('svm', GridSearchCV(svm.SVC(max_iter=100000, probability=True), params)), ])
    svm_pipe_mm.fit(x_train, y_train.loc[:,mood].values)
    svms_mm[mood] = svm_pipe_mm

C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [12]:
# Define functions for prediction and evaluation of SVM model

from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score

# Predict binary values or confidence scores for moods from sensor data
# For now, input_data is a Dataframe for convenience
def svm_predict(svms, input_data, prob=True):
    pred_df = input_data.copy()
    for mood,svm in svms.items():
        if prob:
            pred = svm.predict_proba(pred_df.iloc[:,:-8].values)
            pred_df[mood] = pred[:,1]
        else:
            pred = svm.predict(pred_df.iloc[:,:-8].values)
            pred_df[mood] = pred.reshape(-1)
    return pred_df
    
# Get loss (MSE) of predicted confidence scores
# Input: DataFrames of actual and predicted moods
def evaluate(df_actual, df_pred, df_pred_proba, clf):
    evaluation_df_dict = {
        'mood': [],
        'accuracy (%s)'%clf: [],
        'precision (%s)'%clf: [],
        'recall (%s)'%clf: [],
        'mse (%s)'%clf: [],
    }
    df_moods_actual = df_actual.iloc[:,-8:]
    df_moods_pred = df_pred.iloc[:,-8:]
    df_moods_pred_proba = df_pred_proba.iloc[:,-8:]
    for mood in df_moods_actual.columns:
        y_actual, y_pred, y_pred_proba = df_moods_actual[mood].values, df_moods_pred[mood].values, df_moods_pred_proba[mood].values
        evaluation_df_dict['mood'].append(mood)
        evaluation_df_dict['accuracy (%s)'%clf].append(accuracy_score(y_actual, y_pred))
        evaluation_df_dict['precision (%s)'%clf].append(precision_score(y_actual, y_pred, zero_division=0))
        evaluation_df_dict['recall (%s)'%clf].append(recall_score(y_actual, y_pred))
        evaluation_df_dict['mse (%s)'%clf].append(mean_squared_error(y_actual, y_pred))
    evaluation_df = pd.DataFrame(evaluation_df_dict)
    evaluation_df.set_index('mood', inplace=True)
    return evaluation_df

In [20]:
train_pred_svm_df = svm_predict(svms_std, train_df, prob=False)
test_pred_svm_df = svm_predict(svms_std, test_df, prob=False)
train_prob_svm_df = svm_predict(svms_std, train_df, prob=True)
test_prob_svm_df = svm_predict(svms_std, test_df, prob=True)

# Show predicted values and confidence scores for moods for random samples
print('Test data mood labels (Actual):')
test_df_copy = test_df.copy()
test_df_copy['activity'] = [activity_cats[k] for k in test_df_copy['activity'].values]
display(test_df_copy.head())

print('Test data mood labels (Predicted):')
test_pred_svm_df['activity'] = [activity_cats[k] for k in test_pred_svm_df['activity'].values]
display(test_pred_svm_df.head())

print('Test data mood labels (Confidence Scores):')
test_prob_svm_df['activity'] = [activity_cats[k] for k in test_prob_svm_df['activity'].values]
display(test_prob_svm_df.head())

# Show evaluation of predictions
train_eval_svm_df = evaluate(train_df, train_pred_svm_df, train_prob_svm_df, 'svm,std')
test_eval_svm_df = evaluate(test_df, test_pred_svm_df, test_prob_svm_df, 'svm,std')
display(test_eval_svm_df)
display(test_eval_svm_df.describe())

# # Show MSE of confidence scores
# print('MSE (train): ', evaluate_mse(train_df, train_pred_svm_df))
# print('MSE (test): ', evaluate_mse(test_df, test_pred_svm_df))

# # Show predicted confidence scores for moods for different activities
# print('\nPredicted moods for different activities:')
# for activity in le.classes_:
#     display(test_pred_svm_df[test_pred_svm_df['activity']==activity].head())

Test data mood labels (Actual):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
119,234.80,33.818970,58.093262,2,0,0,0,0,0,1,0,1
91,92.69,34.100952,75.988770,2,0,0,1,0,0,0,0,1
24,60.68,32.630615,61.993408,2,1,1,1,0,1,1,1,1
376,29688.32,38.139343,55.944824,1,0,0,0,0,1,0,0,0
207,5607.68,34.674988,70.166016,1,1,1,1,1,1,1,1,0


Test data mood labels (Predicted):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
119,234.80,33.818970,58.093262,2,0,0,0,0,0,0,0,0
91,92.69,34.100952,75.988770,2,0,0,0,0,0,0,0,0
24,60.68,32.630615,61.993408,2,0,0,0,0,0,0,0,0
376,29688.32,38.139343,55.944824,1,0,0,0,0,0,0,0,0
207,5607.68,34.674988,70.166016,1,0,0,0,1,0,0,0,0


Test data mood labels (Confidence Scores):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
119,234.80,33.818970,58.093262,2,0.360807,0.417752,0.468365,0.418734,0.396548,0.431776,0.480443,0.459029
91,92.69,34.100952,75.988770,2,0.365383,0.394110,0.513586,0.424330,0.474210,0.437722,0.480094,0.465625
24,60.68,32.630615,61.993408,2,0.364670,0.414037,0.480603,0.441373,0.412179,0.432465,0.480389,0.467527
376,29688.32,38.139343,55.944824,1,0.108926,0.407390,0.343670,0.507915,0.396436,0.437082,0.480109,0.525528
207,5607.68,34.674988,70.166016,1,0.420311,0.418618,0.475369,0.530233,0.406451,0.437553,0.479987,0.469492


,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)"
mood,,,,
Aggressive,0.631579,0.818182,0.214286,0.368421
Athletic,0.589474,0.666667,0.340426,0.410526
Atmospheric,0.494737,0.000000,0.000000,0.505263
Celebratory,0.610526,0.656250,0.446809,0.389474
Depressive,0.494737,0.470588,0.347826,0.505263
Elegant,0.452632,0.000000,0.000000,0.547368
Passionate,0.600000,0.461538,0.162162,0.400000
Warm,0.547368,0.666667,0.045455,0.452632


,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)"
count,8.000000,8.000000,8.000000,8.000000
mean,0.552632,0.467486,0.194620,0.447368
std,0.065375,0.310552,0.172285,0.065375
min,0.452632,0.000000,0.000000,0.368421
25%,0.494737,0.346154,0.034091,0.397368
50%,0.568421,0.563419,0.188224,0.431579
75%,0.602632,0.666667,0.342276,0.505263
max,0.631579,0.818182,0.446809,0.547368


In [21]:
train_mm_pred = svm_predict(svms_mm, train_df, prob=False)
test_mm_pred = svm_predict(svms_mm, test_df, prob=False)
train_mm_prob = svm_predict(svms_mm, train_df, prob=True)
test_mm_prob = svm_predict(svms_mm, test_df, prob=True)

# Show evaluation of predictions
train_eval_mm = evaluate(train_df, train_mm_pred, train_mm_prob, 'svm,mm')
test_eval_mm = evaluate(test_df, test_mm_pred, train_mm_prob, 'svm,mm')
display(test_eval_mm)
display(test_eval_mm.describe())

,"accuracy (svm,mm)","precision (svm,mm)","recall (svm,mm)","mse (svm,mm)"
mood,,,,
Aggressive,0.652632,0.800000,0.285714,0.347368
Athletic,0.578947,0.629630,0.361702,0.421053
Atmospheric,0.494737,0.000000,0.000000,0.505263
Celebratory,0.621053,0.657143,0.489362,0.378947
Depressive,0.515789,0.500000,0.326087,0.484211
Elegant,0.452632,0.000000,0.000000,0.547368
Passionate,0.600000,0.466667,0.189189,0.400000
Warm,0.536842,0.000000,0.000000,0.463158


,"accuracy (svm,mm)","precision (svm,mm)","recall (svm,mm)","mse (svm,mm)"
count,8.000000,8.000000,8.000000,8.000000
mean,0.556579,0.381680,0.206507,0.443421
std,0.068088,0.331794,0.190075,0.068088
min,0.452632,0.000000,0.000000,0.347368
25%,0.510526,0.000000,0.000000,0.394737
50%,0.557895,0.483333,0.237452,0.442105
75%,0.605263,0.636508,0.334991,0.489474
max,0.652632,0.800000,0.489362,0.547368


# Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier

# params = {'n_estimators':[10,50,100,150,200,500], 'criterion':('gini', 'entropy'), 'max_depth':[k for k in range(1,21)]}
params = {'criterion':('gini', 'entropy'), 'max_depth':[k for k in range(1,21)]}
rf = GridSearchCV(RandomForestClassifier(random_state=0), params)
rf.fit(x_train, y_train)
rf.best_params_

{'criterion': 'gini', 'max_depth': 15}

In [28]:
# Predict binary values or confidence scores for moods from sensor data
# For now, input_data is a Dataframe for convenience
# Returns confidence scores if prob=True else binary values
def rf_predict(rf, input_data, prob=True):
    pred_df = input_data.copy()
    data = input_data.iloc[:,:-8]
    target = input_data.iloc[:,-8:]
    if prob:
        pred = np.array(rf.predict_proba(data.values))
    else:
        pred = np.array(rf.predict(data.values))
    for i in range(len(target.columns)):
        mood = target.columns[i]
        if prob:
            pred_df[mood] = pred[i,:,1] # predict_proba returns shape (n_features, n_samples, probs)
        else:
            pred_df[mood] = pred[:,i] # predict returns shape (n_samples, n_features)
    return pred_df

In [30]:
train_pred_rf_df = rf_predict(rf, train_df, prob=False)
test_pred_rf_df = rf_predict(rf, test_df, prob=False)
train_prob_rf_df = rf_predict(rf, train_df, prob=True)
test_prob_rf_df = rf_predict(rf, test_df, prob=True)

# Show predicted values and confidence scores for moods for random samples
print('Test data mood labels (Actual):')
test_df_copy = test_df.copy()
test_df_copy['activity'] = [activity_cats[k] for k in test_df_copy['activity'].values]
# test_df_copy['activity'] = le.inverse_transform(test_df_copy['activity'].values) # convert activity values back to strings
display(test_df_copy.head())

print('Test data mood labels (Predicted):')
# test_pred_rf_df['activity'] = le.inverse_transform(test_pred_rf_df['activity'].values)
test_pred_rf_df['activity'] = [activity_cats[k] for k in test_pred_rf_df['activity'].values]
display(test_pred_rf_df.head())

print('Test data mood labels (Confidence Scores):')
test_prob_rf_df['activity'] = [activity_cats[k] for k in test_prob_rf_df['activity'].values]
# test_pred_proba_rf_df['activity'] = le.inverse_transform(test_pred_proba_rf_df['activity'].values)
display(test_prob_rf_df.head())

# Show evaluation of predictions
train_eval_rf_df = evaluate(train_df, train_pred_rf_df, train_prob_rf_df, 'rf')
test_eval_rf_df = evaluate(test_df, test_pred_rf_df, test_prob_rf_df, 'rf')
display(test_eval_rf_df)
display(test_eval_rf_df.describe())

# print('MSE (train): ', evaluate(train_df, train_pred_rf_df))
# print('MSE (test): ', evaluate(test_df, test_pred_rf_df))

# print('\nPredicted moods for different activities:')
# for activity in le.classes_:
#     display(test_pred_rf_df[test_pred_rf_df['activity']==activity].head())

Test data mood labels (Actual):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
119,234.80,33.818970,58.093262,Working,0,0,0,0,0,1,0,1
91,92.69,34.100952,75.988770,Working,0,0,1,0,0,0,0,1
24,60.68,32.630615,61.993408,Working,1,1,1,0,1,1,1,1
376,29688.32,38.139343,55.944824,Walking,0,0,0,0,1,0,0,0
207,5607.68,34.674988,70.166016,Walking,1,1,1,1,1,1,1,0


Test data mood labels (Predicted):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
119,234.80,33.818970,58.093262,Working,0,0,0,0,0,0,1,0
91,92.69,34.100952,75.988770,Working,0,0,1,0,0,0,0,0
24,60.68,32.630615,61.993408,Working,0,1,1,1,1,1,1,1
376,29688.32,38.139343,55.944824,Walking,0,0,0,0,0,0,0,1
207,5607.68,34.674988,70.166016,Walking,1,1,1,1,1,1,1,0


Test data mood labels (Confidence Scores):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
119,234.80,33.818970,58.093262,Working,0.069338,0.384148,0.162805,0.303493,0.292919,0.117036,0.518746,0.323657
91,92.69,34.100952,75.988770,Working,0.107756,0.112744,0.749810,0.061131,0.166048,0.162601,0.207810,0.084554
24,60.68,32.630615,61.993408,Working,0.065603,0.811171,0.908795,0.850068,0.669291,0.827171,0.613346,0.877043
376,29688.32,38.139343,55.944824,Walking,0.010000,0.020000,0.260000,0.130000,0.010000,0.260000,0.130000,0.610000
207,5607.68,34.674988,70.166016,Walking,0.651974,0.603429,0.608244,0.617949,0.604096,0.622577,0.626032,0.340365


,accuracy (rf),precision (rf),recall (rf),mse (rf)
mood,,,,
Aggressive,0.547368,0.485714,0.404762,0.452632
Athletic,0.536842,0.528302,0.595745,0.463158
Atmospheric,0.505263,0.511628,0.458333,0.494737
Celebratory,0.600000,0.591837,0.617021,0.400000
Depressive,0.410526,0.407407,0.478261,0.589474
Elegant,0.463158,0.511628,0.423077,0.536842
Passionate,0.505263,0.395833,0.513514,0.494737
Warm,0.568421,0.529412,0.613636,0.431579


,accuracy (rf),precision (rf),recall (rf),mse (rf)
count,8.000000,8.000000,8.000000,8.000000
mean,0.517105,0.495220,0.513044,0.482895
std,0.060190,0.065264,0.086017,0.060190
min,0.410526,0.395833,0.404762,0.400000
25%,0.494737,0.466138,0.449519,0.447368
50%,0.521053,0.511628,0.495887,0.478947
75%,0.552632,0.528579,0.600218,0.505263
max,0.600000,0.591837,0.617021,0.589474


In [31]:
evaluate_df = pd.concat([test_eval_svm_df, test_eval_mm, test_eval_rf_df], axis=1)
display(evaluate_df)
display(evaluate_df.describe())

,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)","accuracy (svm,mm)","precision (svm,mm)","recall (svm,mm)","mse (svm,mm)",accuracy (rf),precision (rf),recall (rf),mse (rf)
mood,,,,,,,,,,,,
Aggressive,0.631579,0.818182,0.214286,0.368421,0.652632,0.800000,0.285714,0.347368,0.547368,0.485714,0.404762,0.452632
Athletic,0.589474,0.666667,0.340426,0.410526,0.578947,0.629630,0.361702,0.421053,0.536842,0.528302,0.595745,0.463158
Atmospheric,0.494737,0.000000,0.000000,0.505263,0.494737,0.000000,0.000000,0.505263,0.505263,0.511628,0.458333,0.494737
Celebratory,0.610526,0.656250,0.446809,0.389474,0.621053,0.657143,0.489362,0.378947,0.600000,0.591837,0.617021,0.400000
Depressive,0.494737,0.470588,0.347826,0.505263,0.515789,0.500000,0.326087,0.484211,0.410526,0.407407,0.478261,0.589474
Elegant,0.452632,0.000000,0.000000,0.547368,0.452632,0.000000,0.000000,0.547368,0.463158,0.511628,0.423077,0.536842
Passionate,0.600000,0.461538,0.162162,0.400000,0.600000,0.466667,0.189189,0.400000,0.505263,0.395833,0.513514,0.494737
Warm,0.547368,0.666667,0.045455,0.452632,0.536842,0.000000,0.000000,0.463158,0.568421,0.529412,0.613636,0.431579


,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)","accuracy (svm,mm)","precision (svm,mm)","recall (svm,mm)","mse (svm,mm)",accuracy (rf),precision (rf),recall (rf),mse (rf)
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,0.552632,0.467486,0.194620,0.447368,0.556579,0.381680,0.206507,0.443421,0.517105,0.495220,0.513044,0.482895
std,0.065375,0.310552,0.172285,0.065375,0.068088,0.331794,0.190075,0.068088,0.060190,0.065264,0.086017,0.060190
min,0.452632,0.000000,0.000000,0.368421,0.452632,0.000000,0.000000,0.347368,0.410526,0.395833,0.404762,0.400000
25%,0.494737,0.346154,0.034091,0.397368,0.510526,0.000000,0.000000,0.394737,0.494737,0.466138,0.449519,0.447368
50%,0.568421,0.563419,0.188224,0.431579,0.557895,0.483333,0.237452,0.442105,0.521053,0.511628,0.495887,0.478947
75%,0.602632,0.666667,0.342276,0.505263,0.605263,0.636508,0.334991,0.489474,0.552632,0.528579,0.600218,0.505263
max,0.631579,0.818182,0.446809,0.547368,0.652632,0.800000,0.489362,0.547368,0.600000,0.591837,0.617021,0.589474


## Save Model

In [32]:
import pickle
pickle.dump(rf, open('RandomForest', 'wb'))